In [1]:
#Include
using LightGraphs, SparseArrays, SimpleWeightedGraphs
using Statistics, BenchmarkTools, LinearAlgebra, ProgressMeter
using Base.Threads, PhyloNetworks
using Base.GC, JLD2, FileIO
using Random

In [2]:
function tm()
    println("Number of threads = $(nthreads())")

    # sin1(x::Float64) = ccall((:sin, Base.Math.libm), Float64, (Float64,), x)
    # cos1(x::Float64) = ccall((:cos, Base.Math.libm), Float64, (Float64,), x)
    sin1(x::Float64) = ccall(:sin, Float64, (Float64,), x)
    cos1(x::Float64) = ccall(:cos, Float64, (Float64,), x)

    function test1!(y, x)
        # @assert length(y) == length(x)
        for i = 1:length(x)
            y[i] = sin1(x[i])^2 + cos1(x[i])^2
        end
        y
    end

    function testn!(y::Vector{Float64}, x::Vector{Float64})
        # @assert length(y) == length(x)
        Threads.@threads for i = 1:length(x)
            y[i] = sin1(x[i])^2 + cos1(x[i])^2
        end
        y
    end
    n = 10^7
    x = rand(n)
    y = zeros(n)
    @time test1!(y, x)
    @time testn!(y, x)
    @time test1!(y, x)
    @time testn!(y, x);
    flush(stdout)
end

tm()

Number of threads = 8
  0.378807 seconds
  0.201988 seconds (539 allocations: 35.440 KiB)
  1.002391 seconds
  0.228819 seconds (1 allocation: 48 bytes)


In [3]:
function read_tree(filename,c)
    lines = readlines(open(filename))
    count = Dict()
    for i = 1:length(lines)
        l = split(lines[i],c)
        count[l[1]] = 1 
        count[l[2]] = 1
    end
    
    n = length(count)
    G = SimpleGraph(n)
    for i = 1:length(lines)
        l = split(lines[i],c)
        add_edge!(G, parse(Int64,l[1])+1,parse(Int64,l[2])+1)
    end
    
    return G
end

read_tree (generic function with 1 method)

In [4]:
function gid(D,w,x,y)
    return 0.5*(D[w,x]+D[w,y]-D[x,y])
end

function metric_to_structure(d,p2,jj)
    n,_ = size(d)
    global W = zeros(2*n,2*n)
    W[1:n,1:n] = d
    G = SimpleGraph(n)
    
    p = randperm(n)
    
    x = p[1]
    y = p[2]
    z = p[3]
    V = collect(4:n)
    for i = 4:n
        V[i-3] = p[i]
    end
    
    global nextRoots = collect(2*n:-1:n+1)
    
    G= recursive_step(G,V,x,y,z,1)
    
    @show((nv(G),ne(G)))

    for i = 1:nv(G)
        if has_edge(G,i,i)
            rem_edge!(G,i,i)
        end
    end
    
    return G,W
end

function recursive_step(G,V,x,y,z,ztype,tol = 0.000000001)
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = []
    X2 = []
    Y1 = []
    Y2 = []
    Z1 = []
    Z2 = []
    R1 = []
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    for w in V
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1,w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - c) < tol
                push!(Z1,w)
            else
                push!(Z2,w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(W[w,z] - a) < tol || abs(W[w,z] - c) < tol
                push!(X1,w)
            else
                push!(X2,w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - a) < tol
                push!(Y1,w)
            else
                push!(Y2,w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
        
    G = zone1_recurse(G,R1,r)
    G = zone1_recurse(G,X1,x)
    G = zone1_recurse(G,Y1,y)
    G = zone1_recurse(G,Z1,z)
    
    G = zone2_recurse(G,X2,x,r)
    G = zone2_recurse(G,Y2,y,r)
    G = zone2_recurse(G,Z2,z,r)
    
    return G
end

function zone1_recurse(G,V,x,tol = 0.000000001)
    ztype = 1 
    if length(V) == 0
        return G
    end
    
    if length(V) == 1
        add_edge!(G,x,V[1])
        return G
    end
    
    y = V[1]
    z = V[2]
    
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    R1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    Ṽ = V[3:end]
    @inbounds Threads.@threads for w in Ṽ 
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1[Threads.threadid()],w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(W[w,z] - b) < tol && abs(W[w,z] - c) < tol
                push!(Z1[Threads.threadid()],w)
            else
                push!(Z2[Threads.threadid()],w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(W[w,z] - a) < tol && abs(W[w,z] - c) < tol
                push!(X1[Threads.threadid()],w)
            else
                push!(X2[Threads.threadid()],w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(W[w,z] - b) < tol && abs(W[w,z] - a) < tol
                push!(Y1[Threads.threadid()],w)
            else
                push!(Y2[Threads.threadid()],w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
    
    R1p = R1[1]
    X1p = X1[1]
    Y1p = Y1[1]
    Z1p = Z1[1]
    X2p = X2[1]
    Y2p = Y2[1]
    Z2p = Z2[1]
    for i = 2:16
        R1p = append!(R1p,R1[i])
        X1p = append!(X1p,X1[i])
        Y1p = append!(Y1p,Y1[i])
        Z1p = append!(Z1p,Z1[i])
        X2p = append!(X2p,X2[i])
        Y2p = append!(Y2p,Y2[i])
        Z2p = append!(Z2p,Z2[i])
    end
        
        
    G = zone1_recurse(G,R1p,r)
    G = zone1_recurse(G,X1p,x)
    G = zone1_recurse(G,Y1p,y)
    G = zone1_recurse(G,Z1p,z)
    
    G = zone2_recurse(G,X2p,x,r)
    G = zone2_recurse(G,Y2p,y,r)
    G = zone2_recurse(G,Z2p,z,r)
    
    return G
end

function zone2_recurse(G,V,x,y,tol = 0.000000001)
    ztype = 2
    if length(V) == 0
        return G
    end
    
    p = sortperm(W[y,V])

    z = V[p[1]]
    
    n = Int(size(W,1)/2)
    r = pop!(nextRoots)
    add_vertex!(G)
    add_edge!(G,x,r)
    add_edge!(G,y,r)
    add_edge!(G,z,r)
    
    if ztype == 2
        rem_edge!(G,x,y)
    end

    X1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    X2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Y2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    Z2 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    R1 = [[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]]
    
    W[r,x] = gid(W,x,y,z)
    W[x,r] = W[r,x]
    
    replaced_root = false
    
    if abs(W[r,x]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = x
    end
        
    W[y,r] = gid(W,y,x,z)
    W[r,y] = W[y,r]
    
    if abs(W[r,y]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,x,y)
        add_edge!(G,z,y)
        
        push!(nextRoots,r)
        r = y
    end
    
    W[r,z] = gid(W,z,x,y)
    W[z,r] = W[r,z]
    
    if abs(W[r,z]) < tol && !replaced_root
        replaced_root = true
        W[r,x] = 0
        W[x,r] = 0
        W[r,y] = 0
        W[y,r] = 0
        W[r,z] = 0
        W[z,r] = 0
        rem_edge!(G,x,r)
        rem_edge!(G,y,r)
        rem_edge!(G,z,r)
        rem_vertex!(G,r)
        add_edge!(G,z,y)
        add_edge!(G,z,x)
        
        push!(nextRoots,r)
        r = z
    end
    
    Ṽ = V[2:end]
    for i = 2:length(V)
        Ṽ[i-1] = V[p[i]]
    end
    @inbounds Threads.@threads for w in Ṽ 
        a = gid(W,w,x,y)
        b = gid(W,w,y,z)
        c = gid(W,w,z,x)
        
        if abs(a-b) < tol && abs(b-c) < tol && abs(c-a) < tol
            if a < tol && b < tol && c < tol && !replaced_root
                replaced_root = true
                W[w,n+1:2*n] = W[r,n+1:2*n] 
                W[n+1:2*n,w] = W[n+1:2*n,r]
                W[:,r] = zeros(2*n)
                W[r,:] = zeros(2*n)
                rem_edge!(G,x,r)
                rem_edge!(G,y,r)
                rem_edge!(G,z,r)
                rem_vertex!(G,r)
                push!(nextRoots,r)
                r = w
                add_edge!(G,x,r)
                add_edge!(G,y,r)
                add_edge!(G,z,r)
            else
                push!(R1[Threads.threadid()],w)
                W[w,r] = (a+b+c)/3
                W[r,w] = W[w,r]
            end
        elseif a == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - c) < tol
                push!(Z1[Threads.threadid()],w)
            else
                push!(Z2[Threads.threadid()],w)
            end
            W[w,r] = a
            W[r,w] = a
        elseif b == maximum([a,b,c])
            if abs(W[w,z] - a) < tol || abs(W[w,z] - c) < tol
                push!(X1[Threads.threadid()],w)
            else
                push!(X2[Threads.threadid()],w)
            end
            W[w,r] = b
            W[r,w] = b
        elseif c == maximum([a,b,c])
            if abs(W[w,z] - b) < tol || abs(W[w,z] - a) < tol
                push!(Y1[Threads.threadid()],w)
            else
                push!(Y2[Threads.threadid()],w)
            end
            W[w,r] = c
            W[r,w] = c
        end
    end
    
    R1p = R1[1]
    X1p = X1[1]
    Y1p = Y1[1]
    Z1p = Z1[1]
    X2p = X2[1]
    Y2p = Y2[1]
    Z2p = Z2[1]
    for i = 2:16
        R1p = append!(R1p,R1[i])
        X1p = append!(X1p,X1[i])
        Y1p = append!(Y1p,Y1[i])
        Z1p = append!(Z1p,Z1[i])
        X2p = append!(X2p,X2[i])
        Y2p = append!(Y2p,Y2[i])
        Z2p = append!(Z2p,Z2[i])
    end
        
        
    G = zone1_recurse(G,R1p,r)
    G = zone1_recurse(G,X1p,x)
    G = zone1_recurse(G,Y1p,y)
    G = zone1_recurse(G,Z1p,z)
    
    G = zone2_recurse(G,X2p,x,r)
    G = zone2_recurse(G,Y2p,y,r)
    G = zone2_recurse(G,Z2p,z,r)

    return G
end

zone2_recurse (generic function with 2 methods)

In [5]:
function gid_2(D,w,x,y)
    return (D[w,x]+D[w,y]-D[x,y])
end

function metric_to_tree_struct(D,tol=0.005)
    n = size(D,1)
    #Dextended = spzeros(2*n-1,2*n-1)
    #Dextended[1:n,1:n] = D
    G = SimpleGraph(n)
    
    W = zeros(2*n-1,2*n-1)
    
    nextRoots = collect(2*n-1:-1:n+1)
    
    r = pop!(nextRoots)
    add_vertex!(G)
    
    p = ranperm(n)
    
    x=p[1]
    y=p[2]
    z=p[3]
    
    add_edge!(G,r,x)
    add_edge!(G,r,y)
    add_edge!(G,r,z)
    
    W[r,x] = gid_2(D,x,y,z)/2
    W[x,r] = gid_2(D,x,y,z)/2
    
    W[y,r] = gid_2(D,y,x,z)/2
    W[r,y] = gid_2(D,y,x,z)/2
    
    
    W[r,z] = gid_2(D,z,y,x)/2
    W[z,r] = gid_2(D,z,y,x)/2
    
    #Dextended[r,x] = W[r,x]
    #Dextended[x,r] = W[r,x]
    #Dextended[y,r] = W[y,r]
    #Dextended[r,y] = W[y,r]
    #Dextended[z,r] = W[z,r]
    #Dextended[r,z] = W[z,r]
    
    #In Zone type 1
    X1 = []
    Y1 = []
    Z1 = []
    
    #In Zone type 2
    X2 = []
    Y2 = []
    Z2 = []
    
    
    for i = 4:n
        w = p[i]
        a = gid_2(D,w,x,y)/2
        b = gid_2(D,w,x,z)/2
        c = gid_2(D,w,y,z)/2
        
        if a < tol && b < tol && c < tol
            
            #D
            rem_edge!(G,r,x)
            rem_edge!(G,r,y)
            rem_edge!(G,r,z)
            
            W[r,x] = 0
            W[x,r] = 0
    
            W[y,r] = 0
            W[r,y] = 0
    
    
            W[r,z] = 0
            W[z,r] = 0
            
            push!(nextRoots,r)
            rem_vertex!(G,r)
            
            r = w
            
            add_edge!(G,w,x)
            add_edge!(G,w,y)
            add_edge!(G,w,z)
    
            W[w,x] = gid_2(D,x,y,z)/2
            W[x,w] = gid_2(D,x,y,z)/2
    
            W[y,w] = gid_2(D,y,x,z)/2
            W[w,y] = gid_2(D,y,x,z)/2
    
            W[w,z] = gid_2(D,z,y,x)/2
            W[z,w] = gid_2(D,z,y,x)/2
        else
            if a == maximum([a,b,c])
                if b < D[z,w]
                    push!(Z2,(w,b))
                else
                    push!(Z1,w)
                end
                    
            elseif b == maximum([a,b,c])
                if c < D[y,w]
                    push!(Y2,(w,c))
                else
                    push!(Y1,w)
                end
            else
                if a < D[x,w]
                    push!(X2,(w,a))
                else
                    push!(X1,w)
                end
            end
        end
    end
    
    G,W,nextRoots = zone1_recurse_2(G,W,x,X1,nextRoots,D)
    G,W,nextRoots = zone1_recurse_2(G,W,y,Y1,nextRoots,D)
    G,W,nextRoots = zone1_recurse_2(G,W,z,Z1,nextRoots,D)
    
    G,W,nextRoots = zone2_recurse_2(G,W,x,r,X2,nextRoots,D)
    G,W,nextRoots = zone2_recurse_2(G,W,y,r,Y2,nextRoots,D)
    G,W,nextRoots = zone2_recurse_2(G,W,z,r,Z2,nextRoots,D)
            
    
    return G,W
end

function zone1_recurse_2(G,W,x,V,nextRoots,D,tol=0.005)
    n = length(V)
    
    if n == 0
        return G,W,nextRoots
    end
    
    if n == 1
        add_edge!(G,x,V[1])
        W[x,V[1]] = D[x,V[1]]
        W[V[1],x] = D[x,V[1]]
        return G,W,nextRoots
    end
        
    r = pop!(nextRoots)
    add_vertex!(G)
    
    y=V[1]
    z=V[2]
    
    add_edge!(G,r,x)
    add_edge!(G,r,y)
    add_edge!(G,r,z)
    
    W[r,x] = gid_2(D,x,y,z)/2
    W[x,r] = gid_2(D,x,y,z)/2
    
    if W[r,x] == 0.0
        rem_edge!(G,r,x)
        rem_edge!(G,r,y)
        rem_edge!(G,r,z)
        
        rem_vertex!(G,r)
        push!(nextRoots,r)
        
        r = x
        
        add_edge!(G,r,y)
        add_edge!(G,r,z)
    end
    
    W[y,r] = gid_2(D,y,x,z)/2
    W[r,y] = gid_2(D,y,x,z)/2
    
    if W[r,y] == 0.0
        rem_edge!(G,r,x)
        rem_edge!(G,r,y)
        rem_edge!(G,r,z)
        
        rem_vertex!(G,r)
        push!(nextRoots,r)
        
        W[x,y] = W[x,r]
        W[y,x] = W[r,x]
        
        W[x,r] = 0.0
        W[r,x] = 0.0
        
        r = y
        
        add_edge!(G,r,x)
        add_edge!(G,r,z)
    end
    
    W[r,z] = gid_2(D,z,y,x)/2
    W[z,r] = gid_2(D,z,y,x)/2
    
     if W[r,z] == 0.0
        push!(nextRoots,r)
        
        rem_edge!(G,r,z)
        rem_edge!(G,r,x)
        rem_edge!(G,r,y)
        
        rem_vertex!(G,r)
        
        W[r,x] = 0.0
        W[x,r] = 0.0
        W[y,r] = 0.0
        W[r,y] = 0.0
        
        r = z
        
        add_edge!(G,z,x)
        add_edge!(G,z,y)
        
        W[r,x] = D[x,z]
        W[x,r] = D[x,z]
        W[r,y] = D[y,z]
        W[y,r] = D[y,z]
        
    end
        
    
    #In Zone type 1
    X1 = []
    Y1 = []
    Z1 = []
    
    #In Zone type 2
    X2 = []
    Y2 = []
    Z2 = []
    
    for i = 3:n
        w = V[i]
        a = gid_2(D,w,x,y)/2
        b = gid_2(D,w,x,z)/2
        c = gid_2(D,w,y,z)/2
        
        if a < tol && b < tol && c < tol
            numroots = 0
            
            rem_edge!(G,r,x)
            rem_edge!(G,r,y)
            rem_edge!(G,r,z)
            
            W[r,x] = 0
            W[x,r] = 0
    
            W[y,r] = 0
            W[r,y] = 0
    
            W[r,z] = 0
            W[z,r] = 0
            
            rem_vertex!(G,r)
            
            push!(nextRoots,r)
            
            r = w
            
            add_edge!(G,w,x)
            add_edge!(G,w,y)
            add_edge!(G,w,z)
    
            W[w,x] = gid_2(D,x,y,z)/2
            W[x,w] = gid_2(D,x,y,z)/2
    
            W[y,w] = gid_2(D,y,x,z)/2
            W[w,y] = gid_2(D,y,x,z)/2
    
            W[w,z] = gid_2(D,z,y,x)/2
            W[z,w] = gid_2(D,z,y,x)/2
        else
            if a == maximum([a,b,c])
                if b < D[z,w]
                    push!(Z2,(w,b))
                else
                    push!(Z1,w)
                end
                    
            elseif b == maximum([a,b,c])
                if c < D[y,w]
                    push!(Y2,(w,c))
                else
                    push!(Y1,w)
                end
            else
                if a < D[x,w]
                    push!(X2,(w,a))
                else
                    push!(X1,w)
                end
            end
        end
    end
    
    G,W,nextRoots = zone1_recurse_2(G,W,y,Y1,nextRoots,D)
    G,W,nextRoots = zone1_recurse_2(G,W,z,Z1,nextRoots,D)
    
    G,W,nextRoots = zone2_recurse_2(G,W,x,r,X2,nextRoots,D)
    G,W,nextRoots = zone2_recurse_2(G,W,y,r,Y2,nextRoots,D)
    G,W,nextRoots = zone2_recurse_2(G,W,z,r,Z2,nextRoots,D)
    
    return G,W,nextRoots
end

function zone2_recurse_2(G,W,x,y,V,nextRoots,D,tol=0.005)
    n = length(V)
    if n == 0
        return G,W,nextRoots
    end
    
    dxy = W[x,y]
    
    r = pop!(nextRoots)
    add_vertex!(G)
    
    dists = zeros(length(V))
    for i = 1:length(V)
        a,gya = V[i]
        dxa = D[x,a]
        dya = gya + (dxy - (dxa - gya))
        dists[i] = (dxy + dya - dxa)/2
    end
    
    p = sortperm(dists)
        
    
    z,gyz = V[p[1]]
    
    dxz = D[x,z]
    dyz = gyz + (dxy - (dxz - gyz))
    
    
    rem_edge!(G,x,y)
    
    add_edge!(G,r,x)
    add_edge!(G,r,y)
    add_edge!(G,r,z)
    
    W[r,x] = (dxy + dxz - dyz)/2
    W[x,r] = (dxy + dxz - dyz)/2
    
    W[y,r] = (dxy + dyz - dxz)/2
    W[r,y] = (dxy + dyz - dxz)/2
    
    W[r,z] = (dxz + dyz - dxy)/2
    W[z,r] = (dxz + dyz - dxy)/2
    
    if W[r,z] == 0.0
        push!(nextRoots,r)
        
        rem_edge!(G,r,z)
        rem_edge!(G,r,x)
        rem_edge!(G,r,y)
        
        rem_vertex!(G,r)
        
        W[r,x] = 0.0
        W[x,r] = 0.0
        W[y,r] = 0.0
        W[r,y] = 0.0
        
        r = z
        
        add_edge!(G,z,x)
        add_edge!(G,z,y)
        
        W[r,x] = D[x,z]
        W[x,r] = D[x,z]
        W[r,y] = dxy
        W[y,r] = dxy
        
    end
        
    #In Zone type 1
    X1 = []
    Y1 = []
    Z1 = []
    
    #In Zone type 2
    X2 = []
    Y2 = []
    Z2 = []
    
    for i = 2:n
        w,gyw = V[p[i]]
        dxw = D[x,w]
        dzw = D[z,w]
        
        dyw = gyw + (dxy - (dxw - gyw))
        
        a = (dxw+dyw-dxy)/2
        b = (dxw+dzw-dxz)/2
        c = (dyw+dzw-dyz)/2
        
        if a < tol && b < tol && c < tol
            
            rem_edge!(G,r,x)
            rem_edge!(G,r,y)
            rem_edge!(G,r,z)
            
            W[r,x] = 0
            W[x,r] = 0
    
            W[y,r] = 0
            W[r,y] = 0
    
    
            W[r,z] = 0
            W[z,r] = 0
            
            rem_vertex!(G,r)
            push!(nextRoots,r)
            
            r = w
            
            add_edge!(G,w,x)
            add_edge!(G,w,y)
            add_edge!(G,w,z)
    
            W[r,x] = (dxy + dxz - dyz)/2
            W[x,r] = (dxy + dxz - dyz)/2
    
            W[y,r] = (dxy + dyz - dxz)/2
            W[r,y] = (dxy + dyz - dxz)/2
    
            W[r,z] = (dxz + dyz - dxy)/2
            W[z,r] = (dxz + dyz - dxy)/2
        else
            if a == maximum([a,b,c])
                if b < dzw
                    push!(Z2,(w,b))
                else
                    push!(Z1,w)
                end
                    
            #elseif b == maximum([a,b,c])
            #    if c < dyw
            #        @show((a,b,c))
            #        push!(Y2,(w,c))
            #    else
            #        push!(Y1,w)
            #    end
            else
                if a < dxw
                    push!(X2,(w,a))
                else
                    push!(X1,w)
                end
            end
        end
    end
    
    G,W,nextRoots = zone1_recurse_2(G,W,z,Z1,nextRoots,D)
    
    G,W,nextRoots = zone2_recurse_2(G,W,x,r,X2,nextRoots,D)
    G,W,nextRoots = zone2_recurse_2(G,W,y,r,Y2,nextRoots,D)
    G,W,nextRoots = zone2_recurse_2(G,W,z,r,Z2,nextRoots,D)
    
    return G,W,nextRoots
end  

zone2_recurse_2 (generic function with 2 methods)

In [6]:
function MAP(Dnew,G)
    n = nv(G)
    map = 0
    for i = 1:n
        N = neighborhood(G,i,1)
        D = Dnew[:,i]
        p = sortperm(D)
        P = Dict()
        for j = 1:n
            P[p[j]] = j
        end
        d = length(N)
        for j = 1:d
            R = P[N[j]]
            a = Set(N)
            b = Set(p[1:R])
            map += length(intersect(a,b))/(d*R)
        end
    end
    
    return map/n
end

function avg_distortion(Dnew,Dold)
    n,n = size(Dnew)
    d = 0
    for i = 1:n
        for j = 1:i-1
            d += abs(Dnew[i,j]-Dold[i,j])/Dold[i,j]
        end
    end
    
    return 2*d/(n*(n-1))
end

using ProgressMeter

function parallel_dp_shortest_paths(g,distmx::AbstractMatrix{T}) where T <: Real
    n_v = nv(g)
    N = n_v
    p = Progress(N);
    update!(p,0)
    jj = Threads.Atomic{Int}(0)

    l = Threads.SpinLock()

    # TODO: remove `Int` once julialang/#23029 / #23032 are resolved
    dists   = Array{T,2}(undef,(Int(n_v), Int(n_v)))
    #parents = Array{U,2}(undef,(Int(n_v), Int(n_v)))

    state = LightGraphs.dijkstra_shortest_paths(g,[1],distmx)
    dists[1, :] = state.dists 
        
    Threads.@threads for i in 2:n_v
        state = LightGraphs.dijkstra_shortest_paths(g,[i],distmx)
        dists[i, :] = state.dists
        #parents[i, :] = state.parents
        Threads.atomic_add!(jj, 1)
        Threads.threadid() == 1 && update!(p, jj[])
    end

    #result = MultipleDijkstraState(dists, parents)
    return dists
end

parallel_dp_shortest_paths (generic function with 1 method)

In [7]:
G = read_tree("./../hyperbolics-master/data/edges/bio-grid-human.edges",",")
n = nv(G)
E = ne(G)
@show((n,E));
@show(is_connected(G));

(n, E) = (9436, 30917)
is_connected(G) = false


In [8]:
connected_components(G)
add_edge!(G,1,2)
@show(is_connected(G));

is_connected(G) = false


In [9]:
@time A = parallel_dp_shortest_paths(G, adjacency_matrix(G));

Progress:  96%|███████████████████████████████████████▌ |  ETA: 0:00:01

 31.987685 seconds (3.83 M allocations: 9.602 GiB, 5.06% gc time)


In [10]:
global p2 = Progress(nv(G))
global jj = 0

0

In [11]:
times = zeros(20)
map2 = zeros(20)
distort = zeros(20)
@showprogress for j = 1:20
    times[j] = @elapsed G2,W2 = metric_to_structure(A,p2,jj);
    #@show(is_connected(G2),nv(G2),ne(G2));
    B = W2[1:nv(G2),1:nv(G2)];
    B = sparse(B);
    B = (B .> 0) .* B;
    D2 = parallel_dp_shortest_paths(G2, B);

    distort[j] = avg_distortion(D2[1:n,1:n],A)
    map2[j] = MAP(D2[1:n,1:n],G)
    
    @show((distort[j],map2[j]))
end

t = mean(times)
dis = mean(distort)
m = mean(map2)

@show((t,dis,m))

(nv(G), ne(G)) = (11461, 11472)


Progress:  98%|████████████████████████████████████████▏|  ETA: 0:00:01

(distort[j], map2[j]) = (0.40820779011463737, 0.494638114869993)


Progress:   5%|██                                       |  ETA: 0:42:38

(nv(G), ne(G)) = (12335, 12344)


...

Progress:  94%|██████████████████████████████████████▌  |  ETA: 0:00:03

(distort[j], map2[j]) = (0.4126774655209126, 0.5255738760994367)


Progress: 100%|█████████████████████████████████████████| Time: 0:42:38


(t, dis, m) = (31.467209559400004, 0.5310954538114206, 0.5298199737450221)


(31.467209559400004, 0.5310954538114206, 0.5298199737450221)

In [14]:
@time R = nj!(copy(convert(Matrix{Float64},A)));

7079.936381 seconds (975.22 M allocations: 25.679 GiB, 0.12% gc time)


┌ Info: 1 branches had negative lengths, reset to 0
└ @ Main In[13]:133


In [15]:
R

HybridNetwork, Un-rooted Network
18869 edges
18870 nodes: 9436 tips, 0 hybrid nodes, 9434 internal tree nodes.
tip labels: 1, 2, 3, 4, ...
(((((((((((1:0.454,3524:0.546):0.332,(213:0.124,6514:0.876):0.168):0.176,((((650:0.135,7824:0.865):0.191,((968:0.635,(4501:0.274,(5223:0.072,7963:0.928):0.226):0.115):0.276,(3935:0.73,7834:0.27):0.474):0.184):0.127,(690:0.375,(6318:1.0,6319:0.0):0.625):0.373):0.225,((651:0.832,2018:0.168):0.183,7171:0.817):0.252):0.016):0.083,((886:0.276,6436:0.724):0.435,7401:0.565):0.194):0.124,(((587:0.535,(5862:0.449,(((5912:0.0,7492:1.0):0.164,(7490:0.253,7491:0.747):0.336):0.438,6956:0.562):0.051):0.215):0.073,((4648:0.159,4899:0.841):0.315,(((4827:0.0,6518:1.0):0.075,6516:0.925):0.26,6517:0.74):0.185):0.24):0.186,(((2446:0.0,2447:1.0):0.0,7074:1.0):0.829,((2898:0.304,6383:0.696):0.222,7048:0.778):0.171):0.322):0.225):0.122,((((1114:0.639,(1116:0.453,4434:0.547):0.361):0.283,(2438:0.0,6561:1.0):0.217):0.427,6120:0.573):0.459,((2783:0.557,7610:0.443):0.637,(379

In [16]:
g = SimpleGraph(R.numNodes)
w = spzeros(R.numNodes,R.numNodes)
for i = 1:R.numEdges
    src = R.edge[i].node[1].number
    dst = R.edge[i].node[2].number
    add_edge!(g,src,dst)
    w[src,dst] = R.edge[i].length
    w[dst,src] = w[src,dst]
end

In [17]:
@time D3 = parallel_dp_shortest_paths(g, w)

Progress:  98%|████████████████████████████████████████▍|  ETA: 0:00:01

 68.183605 seconds (1.30 M allocations: 32.474 GiB, 21.43% gc time)


18870×18870 Array{Float64,2}:
 0.0      3.65579  4.32061  3.54239  …  1.62642     1.62489     1.61984   
 3.65579  0.0      4.72662  3.9585      2.04253     2.0309      2.03595   
 4.32061  4.72662  0.0      4.62333     2.70736     2.69573     2.70078   
 3.54239  3.9585   4.62333  0.0         1.91597     1.92761     1.92255   
 3.7099   4.16538  4.83021  4.05199     2.13601     2.13449     2.12943   
 3.92926  4.34538  5.0102   4.21882  …  2.30284     2.31448     2.30942   
 3.18624  3.64173  4.30656  3.52833     1.61236     1.61083     1.60578   
 2.57143  3.02692  3.69174  2.91352     0.997546    0.996018    0.990965  
 2.86945  3.32493  3.98976  3.21154     1.29556     1.29404     1.28898   
 2.68562  3.09163  3.72203  2.98834     1.07236     1.06073     1.06578   
 3.44995  3.86607  4.5309   3.73951  …  1.82354     1.83517     1.83012   
 3.00545  3.42156  4.08639  3.295       1.37903     1.39066     1.38561   
 3.811    4.22712  4.89195  4.10056     2.18459     2.19622     2.1911

In [18]:
@show(avg_distortion(D3[1:n,1:n],A));

avg_distortion(D3[1:n, 1:n], A) = 0.09569053555165995


In [19]:
@show(MAP(D3[1:n,1:n],G));

MAP(D3[1:n, 1:n], G) = 0.727817592236963


In [25]:
@time G3,W3 = metric_to_tree_struct(A);
@show(is_connected(G3));
D3 = LightGraphs.Parallel.floyd_warshall_shortest_paths(G3,W3).dists;
@show(avg_distortion(D3[1:n,1:n],A));
@show(MAP(D3[1:n,1:n],G));

UndefVarError: UndefVarError: ranperm not defined

In [ ]:
nthds = nthreads()

X1 = fill([], nthds)

In [ ]:
n,n = size(A)
global d = 0

In [ ]:
p = Progress(n)
for i = 1:n
    for j = 1:i-1
        d += abs(A[i,j]-D2[i,j])/A[i,j]
    end
    next!(p)
end
    
2*d/(n*(n-1))

In [ ]:
map = 0
p2 = Progress(n)
for i = 1:n
    N = neighborhood(G,i,1)
    D = D2[:,i]
    p = sortperm(D)
    P = Dict()
    for j = 1:n
        P[p[j]] = j
    end
    d = length(N)
    for j = 1:d
        R = P[N[j]]
        a = Set(N)
        b = Set(p[1:R])
        map += length(intersect(a,b))/(d*R)        
    end
    next!(p2)
end

map/n

In [13]:
function check_distance_matrix(D::Matrix{<:Real})
    size(D, 1) > 1 || throw(DomainError(D, "Too few nodes"))
    if any(D .< 0)
        throw(DomainError(D, "Negative entries in distance matrix"))
    end
    if any(diag(D) .!= 0.0)
        throw(DomainError(D, "Diagonal entry not 0"))
    end
end


"""
    nj!(D::Matrix{Float64}, names::AbstractVector{String}=String[];
        force_nonnegative_edges::Bool=false)
Construct a phylogenetic tree from the input distance matrix and
vector of names (as strings), using the Neighbour-Joinging algorithm
([Satou & Nei 1987](https://doi.org/10.1093/oxfordjournals.molbev.a040454)).
The order of the `names` argument should match that of the row (and
column) of the matrix `D`.
With `force_nonnegative_edges` being `true`, any negative edge length
is changed to 0.0 (with a message).
Warning: `D` is modified.
"""
function nj!(D::Matrix{Float64}, names::AbstractVector{String}=String[];
             force_nonnegative_edges::Bool=true)

    check_distance_matrix(D)
    n = size(D, 1)              # number of species

    # when no names arg is supplied
    if isempty(names)
        names = string.(1:n)
    end

    # create empty network with n unconnected leaf nodes
    nodes = map(function(i)
                node = PhyloNetworks.Node(i, true)
                node.name = names[i]
                return node
                end,
                1:n)

    net = PhyloNetworks.HybridNetwork(nodes, PhyloNetworks.Edge[])
    # an array of Node s.t. active_nodes[i] would correspond to the
    # ith entry in distance matrix D at each iteration
    active_nodes = nodes

    neglenp = 0  # number of negative edge lengths

    while n > 2
        # compute Q matrix and find min
        # warning: here i and j are indeces for D, not actual id's
        sums = sum(D, dims = 1)
        cur = Inf
        min_index = (0, 0)
        for i = 1:n
            for j = i:n
                if j != i
                    qij = (n-2) * D[i,j] - sums[i] - sums[j]
                    if qij < cur
                        cur = qij
                        min_index = (i, j)
                    end
                end
            end
        end

        # connect the nodes, compute the length of the edge
        (i, j) = min_index
        dik = D[i,j] / 2 + (sums[i] - sums[j]) / (2 * (n - 2))
        djk = D[i,j] - dik
        # force negative lengths to 0, if any
        if dik < 0.0
            neglenp += 1
            if force_nonnegative_edges dik = 0.0; end
        end
        if djk < 0.0
            neglenp += 1
            if force_nonnegative_edges djk = 0.0; end
        end

        # create new edges and node, update tree
        edgenum = net.numEdges
        eik = PhyloNetworks.Edge(edgenum + 1, dik) # edge length must be Float64 for Edge()
        ejk = PhyloNetworks.Edge(edgenum + 2, djk)
        node_k = PhyloNetworks.Node(net.numNodes+1, false, false, [eik, ejk]) # new node
        node_i = active_nodes[i]
        node_j = active_nodes[j]
        PhyloNetworks.setNode!(eik, PhyloNetworks.Node[node_i, node_k])
        PhyloNetworks.setNode!(ejk, PhyloNetworks.Node[node_j, node_k])
        PhyloNetworks.setEdge!(node_i, eik)
        PhyloNetworks.setEdge!(node_j, ejk)
        PhyloNetworks.pushEdge!(net, eik)
        PhyloNetworks.pushEdge!(net, ejk)
        PhyloNetworks.pushNode!(net, node_k)

        # update map and D
        # replace D[l, i] with D[l, k], delete D[ , j]
        for l in 1:n
            if !(l in [i j])
                D[l, i] = (D[l,i] + D[j,l] - D[i,j]) / 2
                D[i, l] = D[l, i]
            end
        end
        newidx = filter(u->u!=j, 1:n) # index 1:n\{j}
        D = view(D, newidx, newidx)

        # update active_nodes
        active_nodes[i] = node_k
        active_nodes = view(active_nodes, newidx)

        n = n - 1
    end

    # base case
    if D[1,2] < 0.0
        neglenp += 1
        if force_nonnegative_edges D[1,2] = 0.0; end
    end
    node1 = active_nodes[1]
    node2 = active_nodes[2]
    newedge = PhyloNetworks.Edge(net.numEdges+1, D[1,2])
    PhyloNetworks.setNode!(newedge, [node1, node2])
    PhyloNetworks.setEdge!(node1, newedge)
    PhyloNetworks.setEdge!(node2, newedge)
    PhyloNetworks.pushEdge!(net, newedge)

    # report on number of negative branches
    if neglenp > 0
        infostr = (force_nonnegative_edges ?
                   "$neglenp branches had negative lengths, reset to 0" :
                   "$neglenp branches have negative lengths" )
        @info infostr
    end

    return net
end


nj!

In [ ]:
using JLD2, FileIO
@load ("./l1_tree_yeast-1e-5-heurestic.jld2") R

In [ ]:
D4 = R[end]+A;

In [ ]:
@show(MAP(D4[1:n,1:n],G));

In [ ]:
@show(avg_distortion(D4[1:n,1:n],A));

In [20]:
1

1